In [ ]:
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
# Enable interactive plot
%matplotlib notebook
import os
from numpy import linalg as la
import cmath
import random
import sys
import pickle
from scipy.optimize import minimize
from sympy import solve, sqrt, exp
import pandas as pd
import gc
gc.collect()

# Environmental Fitness Function

In [ ]:
# Environmental fitness function with fixed area
def Fitness(x,x0,s):
    #fitness of x in env (x0,s)
    return np.exp(-(x-x0)**2/(2*s**2))/np.sqrt(2*np.pi*s**2)

# Solver for Optimal Phenotype

In [ ]:
# Function to check whether phenotype z is the optimal phenotype. Returns zero when the slopes of the adaptive function
# and the fitness sets match, nonzero value otherwise
def Roots(z,x1,s1,x2,s2,p):
    x = z[0]   
    f = np.empty((1))
    f[0] = p*(x-x1)*s2**2*Fitness(x,x1,s1)+(1-p)*(x-x2)*s1**2*Fitness(x,x2,s2)
    return f


# Solver for optimal phenotype that returns success=1 if the long time optimal is the moving specialist for a 
# given v, p and s2/s1, otherwise returns success=0 
def optp(v,T,p):
    optimalmat=np.zeros((T))
    opt=(x01+x02)/2

    for t in range(T):
        x1=x01
        x2=x02+v*t         

        # The solver is sensitive to initial guesses, so we compare two solutions: one (zGuess1) where the initial guess
        # is one of the specialists, and another (zGuess2) where the initial guess is the optimal solution in the
        # previous time step. This produces the best results, especially for skewed fitness sets.

        m=(Fitness(x2,x2,s2)-Fitness(x1,x2,s2))/(Fitness(x2,x1,s1)-Fitness(x1,x1,s1)) #slope of line joining the two specialists on the fitness set
        mp=-p/(1-p)    #slope of the adaptive function
        if(mp<=m):
            zGuess1 = np.array([x1])         # if the slope of the adaptive function is smaller that that of the 
        elif(mp>m):                          # line joining the two specialists on the fitness set, choose x1 as the initial guess, 
            zGuess1 = np.array([x2])         # otherwise choose x2

        bounds = [(x1,x2)]                   # only look for solutions in the range(x1,x2)
        
        # Roots(z,x1,s1,x2,s2,p) returns a value f for each z, the following finds z within the defined bounds 
        # that minimizes |f|, starting with some initial guesses for z
        z1 = minimize(lambda z: np.linalg.norm(Roots(z,x1,s1,x2,s2,p)),tol=1e-15, x0=zGuess1, bounds=bounds)
        opt1=z1.x[0]
        A1=p*Fitness(opt1,x1,s1)+(1-p)*Fitness(opt1,x2,s2) # value of Adaptive function at the first solution, opt1

        #repeat for second initial guess
        zGuess2 = np.array([opt])
        bounds = [(x1,x2)]
        z2 = minimize(lambda z: np.linalg.norm(Roots(z,x1,s1,x2,s2,p)),tol=1e-15, x0=zGuess2, bounds=bounds)
        opt2=z2.x[0]
        A2=p*Fitness(opt2,x1,s1)+(1-p)*Fitness(opt2,x2,s2)  # value of Adaptive function at the second solution, opt2

        if(A2>A1):         # chose opt2 as the correct optimal if A2>A1, else choose opt1
            opt=opt2
        else:
            opt=opt1

        if abs(x1-opt)<0.5:
            success=0
            break
        elif abs(x2-opt)<0.5:
            success=1
            break
    return success

In [ ]:
Nv=3
v_vals=np.array([0.25,0.5,0.75])    # range of values for the rate of environmental change v

Ns=50
sr0=0.5
ds=0.05
s_range=np.arange(sr0,sr0+Ns*ds,ds)  # range of tolerance asymmetry ratios s2/s1 for the two environments
print(s_range)

s1=10                  # Tolerance of the stationary environment
s2=0                   # Initialize the tolerance of the moving environment

T=500                  # Maximum simulation duration
x01=100                # Initial phenotypic value of stationary specialist
x02=110                # Initial phenotypic value of moving specialist

p_c=np.zeros((Nv,Ns))

dir1='DATA_pc/'
if not os.path.exists(dir1):
    os.mkdir(dir1)

f1=open(dir1+'pc_one_moving.txt',"w")

for vv in range(Nv):
    v=v_vals[vv]
    print("##################### v = {} #####################".format(v))
    for i in range(Ns):
        sr=s_range[i]        
        s2=sr*s1
        
        # Compute p_c using a binary search algorithm
        start=0.0
        end=1.0
        p_range=end-start
        l=0
        while(p_range>1e-4):
            p=start+(end-start)/2
            success=optp(v,T,p)
            #print(l,start,end,p,success)
            if success:
                start=p
            else:
                end=p

            p_range=end-start
            l=l+1
        p_c[vv,i]=start
        print("------- sr = {}, pc = {} -------".format(sr,p_c[vv,i]))
        f1.write("{} {} {}\n".format(v_vals[vv],s_range[i],p_c[vv,i]))
        f1.flush()
f1.close()
pickle.dump(p_c,open(dir1+'pc.p','wb'))